In [ ]:
import pandas as pd
import numpy as np
###Datei einlesen und auf den zweiten Reiter gehen
data = pd.read_excel('Kopie von kw2040r_Awareness_P.xlsx', sheet_name = 1, header=None)
data.rename(columns={0: 'SpalteA'}, inplace=True)
### Herausfinden, welchen Zeilenindex die Zeilen mit Tab. #1 haben, um den oberen Frame zu erhalten
tab_loc = data.loc[data.SpalteA.str.contains('Tab.*') == True,:].index.values
### Herausfinden, welchen Zeilenindex die Zeilen mit Gewichtet mit Weight haben, um den unteren Frame zu erhalten
keine_angabe_loc = data.loc[data.SpalteA.str.contains('Gewichtet mit WEIGHT') == True,:].index.values

###Wie viele Tab Einträge gibt es?
anzahl_frames = int(data.SpalteA.str.contains('Tab. #').sum())

new_df = []
x=0
new_df = []
new_dff = []
df_T = []
###Slicen mittels oberer und unterer Indizes
###data.loc[tab_loc[7]:keine_angabe_loc[7],:]
###Funktion, welche mir entsprechend der Anzahl die Frames zurecht schneidet
### der erste Datenframe wird oben mit tab_loc[0] und unten mit keine_angabe_loc[0] begrenzt
while (x < anzahl_frames):
    ###Frage herausfiltern und speichern
    frage = data.iloc[tab_loc[x]-3,0]
    ###Erstellung neues Datenframes mit Schnittpunkten Tab und "gewichtet mit" für unten
    pd.DataFrame(new_df.append(data.loc[tab_loc[x]:keine_angabe_loc[x],:]))
    ###Löschen der ersten Zeile und der letzten 4
    pd.DataFrame(new_dff.append(new_df[x].iloc[1:-4,:]))
    ###Spalte und Zeilen getauscht mit .T und wieder in die Liste gehängt
    pd.DataFrame(df_T.append(new_dff[x].T))
    ###Header ersetzt durch die Überschriften und neue Überschrift Datum gesetzt und eine unnötige Spalte raus
    header = list(df_T[x].iloc[0,:])
    header[0] = 'Datum'
    header[1] = 'raus'
    df_T[x].columns = header
    df_T[x].drop('SpalteA', axis=0, inplace=True)
    df_T[x].drop('raus', axis=1, inplace=True)
    ###Bereinigung Datum - durch bis ersetzt. Es sind rollierende Wochen, daher lohnt sich die umrechnung in to_date nicht
    df_T[x].Datum.str.replace('-',' bis ', regex=True)
    ### - ersetzt durch isnull 'NaN'
    df_T[x].replace('-', np.nan, inplace=True)
    df_T[x] = df_T[x].assign(Frage=frage) 
    x = x+1

df_final = []
y=0
while (y < anzahl_frames):
    save_columnnames_tmp = df_T[y].columns.tolist()
    save_columnnames_slice = save_columnnames_tmp[1:-1]
    df_final.append(df_T[y].melt(id_vars=['Datum', 'Frage'], value_vars=save_columnnames_slice, var_name='Anbieter'))
    y = y+1
df_excel = pd.DataFrame(pd.concat(df_final))
df_excel.to_excel("RelevantSet1Choice_Markenwerte.xlsx", sheet_name='Markenwerte bereinigt', index=False)  